In [11]:
import pandas as pd
import numpy as np
import glob
import geopandas as gpd
import matplotlib.pyplot as plt
import geoplot as gplt
import shapefile
import osr
import dbf
import requests
import io
import datetime

from urllib.request import urlopen
from zipfile import ZipFile
from shapely.geometry import shape, Point, Polygon


%matplotlib inline

In [ ]:
countypairs = pd.read_csv('/home/jinli/PycharmProjects/county-pair-list.txt')
countypairs.drop_duplicates(subset='COUNTYPAIR_ID', inplace = True)

new = countypairs['COUNTYPAIR_ID'].str.split("-", n = 1, expand = True)

pairid = pd.concat([new[0], new[1]], ignore_index=True)
pairid = pairid.drop_duplicates()
pairid = pairid.to_frame('id')
pairid = pairid[~pairid['id'].isin(['06001', '06041', '06081','06075'])] ### county fips changed

### LAUS data (Unemployment Rate)

In [ ]:
files = glob.glob('/home/jinli/Desktop/Thesis/Data/LAU(unemployment_rate)/*.csv')
dfs = [pd.read_csv(file) for file in files]
lausdata = pd.concat(dfs, ignore_index=True)

lausdata['GEOID10'] = lausdata['Series ID'].map(lambda x: x[5:10])
lausdata['STATEFP10'] = lausdata['GEOID10'].map(lambda x: x[0:2])
lausdata['COUNTYFP10'] = lausdata['GEOID10'].map(lambda x: x[2:])

lausdata = pd.merge(pairid, lausdata, how='left', left_on='id', right_on='GEOID10')
lausdata = lausdata.dropna()
### a = lausdata.GEOID10.unique()
### len(a)      1126

In [ ]:
lausdata.to_csv('LAUS_COUNTY_MONTHLY_UNEMPLOYMENT.csv', index=False)

### Transform monthly LAUS data to quarterly data

In [51]:
mlaus = pd.read_csv('LAUS_COUNTY_MONTHLY_UNEMPLOYMENT.csv')

mlaus['id'] = mlaus['id'].astype(str)
mlaus['id'] = mlaus['id'].str.zfill(5)

mlaus['Year'] = mlaus['Year'].astype(int)
mlaus = mlaus[(mlaus['Year'] > 2004) & (mlaus['Year'] < 2016)]

mlaus['Value'] = pd.to_numeric(mlaus['Value'], errors='coerce')
mlaus = mlaus.replace(np.nan, 0, regex=True)

mlaus['Month'] = mlaus['Period'].str[1:]
mlaus['Year-Month'] = pd.to_datetime(mlaus[['Year', 'Month']].assign(Day=1)).dt.to_period('M')
mlaus['Qtr'] = pd.to_datetime(mlaus[['Year', 'Month']].assign(Day=1)).dt.quarter

mlaus = mlaus[['id', 'Year', 'Qtr', 'Value']]
mlaus.sort_values(by =['id', 'Year', 'Qtr'], ignore_index=True, inplace=True)

qlaus = mlaus.groupby(['id', 'Year', 'Qtr'])['Value'].mean().reset_index()

/home/jinli/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
qlaus.to_csv('LAUS_COUNTY_QUARTERLY_UNEMPLOYMENT.csv', index=False)

### GeoPlot: all states, all counties and broder counties

In [ ]:
## All counties
allcounties = ZipFile('/home/jinli/PycharmProjects/tl_2010_us_county10(NEW).zip', 'r')

filenames_ac = [y for y in sorted(allcounties.namelist())
                 for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]

dbf_ac, prj_ac, shp_ac, shx_ac = [io.BytesIO(allcounties.read(filename)) for filename in filenames_ac]

r_ac = shapefile.Reader(shp=shp_ac, shx=shx_ac, dbf=dbf_ac)
## r_ac.numRecords   ### 3221


attributes, geometry = [], []

field_names = [field[0] for field in r_ac.fields[1:]]

for row in r_ac.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names,row.record)))
    
prj = io.TextIOWrapper(prj_ac, encoding='utf-8')
proj4 = osr.SpatialReference(prj.read()).ExportToProj4()

gdf_ac = gpd.GeoDataFrame(data=attributes, geometry=geometry, crs=proj4)
gdf_ac.sort_values(by =['STATEFP10', 'COUNTYFP10'], inplace=True)
gdf_ac.reset_index(drop=True, inplace=True)
gdf_ac[['INTPTLON10', 'INTPTLAT10']] = gdf_ac[['INTPTLON10', 'INTPTLAT10']].apply(pd.to_numeric)

gdf_ac = gdf_ac[(gdf_ac.STATEFP10 != '02') & (gdf_ac.STATEFP10 != '72') & (gdf_ac.STATEFP10 != '15')]

In [ ]:
## All states
allstates = ZipFile('/home/jinli/PycharmProjects/tl_2010_us_state10.zip', 'r')

filenames_as = [y for y in sorted(allstates.namelist())
                for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]

dbf_as, prj_as, shp_as, shx_as = [io.BytesIO(allstates.read(filename)) for filename in filenames_as]

r_as = shapefile.Reader(shp=shp_as, shx=shx_as, dbf=dbf_as)

attributes, geometry = [], []

field_names = [field[0] for field in r_as.fields[1:]]
for row in r_as.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names,row.record)))
    
prj = io.TextIOWrapper(prj_as, encoding='utf-8')
proj4 = osr.SpatialReference(prj.read()).ExportToProj4()

gdf_as = gpd.GeoDataFrame(data=attributes, geometry=geometry, crs=proj4)
gdf_as = gdf_as[~gdf_as['STATEFP10'].isin(['02', '72', '15'])]
gdf_as.sort_values(by ='STATEFP10', ignore_index=True, inplace=True)

In [ ]:
cp_list = lausdata.GEOID10.tolist()
cp_list = list(set(cp_list)) ## remove duplicates
#cp_list = [e for e in cp_list if e not in ('06001', '06041', '06081','06075')]

In [ ]:
len(cp_list)

In [ ]:
gdf_cp = gdf_ac[gdf_ac['GEOID10'].isin(cp_list)]

In [ ]:
fig, ax = plt.subplots(figsize=(50,50))
gdf_cp.plot(ax=ax, color='steelblue', edgecolor='none')
gdf_ac.plot(ax=ax, facecolor='none', linewidth=0.2, edgecolor='grey')
gdf_as.plot(ax=ax, facecolor='none', linewidth=1, edgecolor='black')

fig.savefig('full_figure.pdf')

### QCEW (Quarterly Census of Employment and Wages)

In [ ]:
ids = pd.read_csv('Paired_County_ID.csv')
ids['id'] = ids['id'].astype(str)
ids['id'] = ids['id'].str.zfill(5)

In [ ]:
files = glob.glob('/home/jinli/Desktop/Thesis/Data/QCEW/2005.q1-q4.by_area/*.csv')
dfs = [pd.read_csv(file) for file in files]
qcewdata = pd.concat(dfs, ignore_index=True)

In [ ]:
qcewdata05 = qcewdata[qcewdata['area_fips'].apply(lambda x: str(x).isdigit())] 

In [ ]:
qcewdata05['area_fips'] = qcewdata05['area_fips'].astype(str)
qcewdata05['area_fips'] = qcewdata05['area_fips'].str.zfill(5)

In [ ]:
qcewdata05 = pd.merge(qcewdata05, ids, how='left', left_on='area_fips', right_on='id')

In [30]:
df01001 = pd.read_csv('/home/jinli/Desktop/Thesis/Data/QCEW/2005.q1-q4.by_area/2005.q1-q4 01001 Autauga County, Alabama.csv')
df01001['area_fips'] = df01001['area_fips'].astype(str)
df01001['area_fips'] = df01001['area_fips'].str.zfill(5)
df01001.drop(df01001.columns[21:], axis=1, inplace=True)
df01001 = df01001.loc[df01001['own_code']==0]

In [31]:
df01001

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,area_title,own_title,...,agglvl_title,size_title,qtrly_estabs_count,month1_emplvl,month2_emplvl,month3_emplvl,total_qtrly_wages,taxable_qtrly_wages,qtrly_contributions,avg_wkly_wage
0,01001,0,10,70,0,2005,1,NaN,"Autauga County, Alabama",Total Covered,...,"County, Total Covered",All establishment sizes,865,11426,11413,11484,78680645,43956791,882017,529
1,01001,0,10,70,0,2005,2,NaN,"Autauga County, Alabama",Total Covered,...,"County, Total Covered",All establishment sizes,869,11520,11575,11498,77222663,17709878,315548,515
2,01001,0,10,70,0,2005,3,NaN,"Autauga County, Alabama",Total Covered,...,"County, Total Covered",All establishment sizes,873,11288,11351,11321,79221594,10058499,167288,538
3,01001,0,10,70,0,2005,4,NaN,"Autauga County, Alabama",Total Covered,...,"County, Total Covered",All establishment sizes,876,11465,11559,11526,80790956,8834411,151831,540


### QWI (Quarterly Workforce Indicators)

In [3]:
#/home/jinli/Desktop/Thesis/Data/QWI/wi_ak_sa_f_gc_ns_op_u.csv.gz
df = pd.read_csv('/home/jinli/Desktop/Thesis/Data/QWI/qwi_ak_sa_f_gc_ns_op_u.csv.gz', 
                 compression='gzip')

In [4]:
df

,periodicity,seasonadj,geo_level,geography,ind_level,industry,ownercode,sex,agegrp,race,...,sFrmJbC,sFrmJbGnS,sFrmJbLsS,sFrmJbCS,sEarnS,sEarnBeg,sEarnHirAS,sEarnHirNS,sEarnSepS,sPayroll
0,Q,U,C,2013,A,00,A05,0,A00,A0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
1,Q,U,C,2013,A,00,A05,0,A01,A0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
2,Q,U,C,2013,A,00,A05,0,A02,A0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,Q,U,C,2013,A,00,A05,0,A03,A0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
4,Q,U,C,2013,A,00,A05,0,A04,A0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918238,Q,U,S,2,S,81,A05,2,A04,A0,...,-1,-1,-1,-1,-1,1,-1,-1,-1,1
918239,Q,U,S,2,S,81,A05,2,A05,A0,...,-1,-1,-1,-1,-1,1,-1,-1,-1,1
918240,Q,U,S,2,S,81,A05,2,A06,A0,...,-1,-1,-1,-1,-1,1,-1,-1,-1,1
918241,Q,U,S,2,S,81,A05,2,A07,A0,...,-1,-1,-1,-1,-1,1,-1,-1,-1,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918243 entries, 0 to 918242
Data columns (total 80 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   periodicity    918243 non-null  object 
 1   seasonadj      918243 non-null  object 
 2   geo_level      918243 non-null  object 
 3   geography      918243 non-null  int64  
 4   ind_level      918243 non-null  object 
 5   industry       918243 non-null  object 
 6   ownercode      918243 non-null  object 
 7   sex            918243 non-null  int64  
 8   agegrp         918243 non-null  object 
 9   race           918243 non-null  object 
 10  ethnicity      918243 non-null  object 
 11  education      918243 non-null  object 
 12  firmage        918243 non-null  int64  
 13  firmsize       918243 non-null  int64  
 14  year           918243 non-null  int64  
 15  quarter        918243 non-null  int64  
 16  Emp            726670 non-null  float64
 17  EmpEnd         726513 non-nul

In [7]:
df.year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016])